<a href="https://colab.research.google.com/github/makarandv/music-emotion-recognition-/blob/master/song_comparison_using_MIDI_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mido

In [3]:
!pip install aubio

     |████████████████████████████████| 481kB 3.3MB/s 
  Created wheel for aubio: filename=aubio-0.4.9-cp36-cp36m-linux_x86_64.whl size=386261 sha256=d9709b1d126296d7341a88bafccd439442f964362ba063db3e5940e44cd89316
  Stored in directory: /root/.cache/pip/wheels/ac/93/da/33a0041dff56d6c1faf82c83ff487cf669442b7087db20a995
Successfully built aubio


In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


The code below is to read the files from the folder 

In [5]:
'''
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/")
'''
import sys
from aubio import source, notes
from mido import Message, MetaMessage, MidiFile, MidiTrack, second2tick, bpm2tempo
import scipy.io.wavfile as wav

if len(sys.argv) < 3:
    print("Usage: %s <filename> <output> [samplerate]" % sys.argv[0])
    sys.exit(1)

    
filename = sys.argv[1]
midioutput = sys.argv[2]

downsample = 1
samplerate = 44100 // downsample
if len( sys.argv ) > 3: samplerate = int(sys.argv[3])

win_s = 512 // downsample # fft size
hop_s = 256 // downsample # hop size

import glob 
files = glob.glob('/content/drive/My Drive/testsamples/*')
print(files)
totd=[]
file_path = '/content/drive/My Drive/testsamples/vandematramlata.wav'
#file_path = "'files(0)'"
(source_rate, source_sig) = wav.read(file_path)
duration_seconds = len(source_sig) / float(source_rate)
totd.append(duration_seconds)
file_path = '/content/drive/My Drive/testsamples/vandematramshreya.wav'
#file_path = "'files(0)'"
(source_rate, source_sig) = wav.read(file_path)
duration_seconds = len(source_sig) / float(source_rate)
totd.append(duration_seconds)
file_path = '/content/drive/My Drive/testsamples/vandematramanuja.wav'
#file_path = "'files(0)'"
(source_rate, source_sig) = wav.read(file_path)
duration_seconds = len(source_sig) / float(source_rate)
totd.append(duration_seconds)
print(len(files))
no= len(files)
mydata2= [[]]
test1=[[]]
for i in range(no):
	mydata2.append([])
for i in range(no):
  test1.append([])

def frames2tick(frames, samplerate=samplerate):
      sec = frames / float(samplerate)
      return int(second2tick(sec, ticks_per_beat, tempo))
 

mydata = []
mydata1= []

c=0
for e in files : 
  print(e)
  s = source(e , samplerate, hop_s)
  samplerate = s.samplerate
  tolerance = 0.8
  notes_o = notes("default", win_s, hop_s, samplerate)
  #
  # create a midi file
  mid = MidiFile()
  track = MidiTrack()
  mid.tracks.append(track)
  ticks_per_beat = mid.ticks_per_beat # default: 480
  bpm = 120 # default midi tempo
  tempo = bpm2tempo(bpm)
  track.append(MetaMessage('set_tempo', tempo=tempo))
  track.append(MetaMessage('time_signature', numerator=4, denominator=4))
  last_time = 0
  total_frames = 0
  t1 = []
  while True:
        samples, read = s()
        new_note = notes_o(samples)
        if (new_note[0] != 0):
          note_str = ' '.join(["%.2f" % i for i in new_note])
          d = new_note;
          print("%.6f" % (total_frames/float(samplerate)), d)
          e1 = []
          for e in d : 
            e1.append(int(e))
            t1.append((total_frames/float(samplerate), e1[:]))
            delta = frames2tick(total_frames) - last_time
            last_time = frames2tick(total_frames)
          test1[c].append (float("%.6f" % (total_frames/float(samplerate))))
          test1[c].append (int(new_note[0]))
          test1[c].append (int(new_note[1]))
          test1[c].append(int(new_note[2]))
        total_frames += read
        if read < hop_s: break
  #mydata.append(t1)
  mydata2[c].append([t1])
  c=c+1

['/content/drive/My Drive/testsamples/vandematramlata.wav', '/content/drive/My Drive/testsamples/vandematramshreya.wav', '/content/drive/My Drive/testsamples/vandematramanuja.wav']
3
/content/drive/My Drive/testsamples/vandematramlata.wav
0.278639 [ 57. 105.  39.]
0.325079 [ 58. 106.  57.]
0.574694 [ 62. 117.  58.]
2.583220 [ 69. 118.  62.]
3.779048 [ 70. 119.  69.]
5.787574 [ 61. 114.   0.]
/content/drive/My Drive/testsamples/vandematramshreya.wav
0.150930 [61. 88.  0.]
0.185760 [60. 81. 61.]
0.226395 [58. 83. 60.]
0.278639 [58. 93. 58.]
0.330884 [60. 94. 58.]
0.679184 [66. 99. 60.]
3.082449 [ 72. 105.  66.]
5.433469 [ 72. 107.   0.]
6.635102 [62. 98.  0.]
/content/drive/My Drive/testsamples/vandematramanuja.wav
1.927256 [ 59. 112.  -1.]
2.304580 [ 63. 112.   0.]
4.806531 [ 72. 111.  63.]
8.504308 [ 62. 108.   0.]
8.655238 [ 60. 103.   0.]


In [6]:
print(test1)

[[0.278639, 57, 105, 39, 0.325079, 58, 106, 57, 0.574694, 62, 117, 58, 2.58322, 69, 118, 62, 3.779048, 70, 119, 69, 5.787574, 61, 114, 0], [0.15093, 61, 88, 0, 0.18576, 60, 81, 61, 0.226395, 58, 83, 60, 0.278639, 58, 93, 58, 0.330884, 60, 94, 58, 0.679184, 66, 99, 60, 3.082449, 72, 105, 66, 5.433469, 72, 107, 0, 6.635102, 62, 98, 0], [1.927256, 59, 112, -1, 2.30458, 63, 112, 0, 4.806531, 72, 111, 63, 8.504308, 62, 108, 0, 8.655238, 60, 103, 0], []]


In [7]:
print (test1[0])

[0.278639, 57, 105, 39, 0.325079, 58, 106, 57, 0.574694, 62, 117, 58, 2.58322, 69, 118, 62, 3.779048, 70, 119, 69, 5.787574, 61, 114, 0]


In [9]:
no=0
print(test1[no])
n=len(test1[no])

# to find the reference note find the duration of each note
i=0
j=4
dur0=[]

print("duration of notes")
while(j<n):
  time = test1[no][j]- test1[no][i]
  dur0.append(round(time,6))
  i=i+4
  j=j+4
temp= totd[no]-test1[no][i]

dur0.append(round(temp,6))
print(dur0)
#identify the max duration and corrosponding note
maxd=0
j=1
k=len(dur0)
while j<k:
  if (dur0[maxd]==dur0[j]):
    if (test1[no][maxd*4+2]<test1[no][j*4+2]):
      print(test1[no][maxd*4+2])
      print(test1[no][j*4+2])
      maxd=j
  elif (dur0[maxd]<dur0[j]):
    maxd=j
  j=j+1

print(maxd)
# display reference note
print(test1[no][(maxd)*4+1])
refnote= test1[no][(maxd)*4+1]
# create relative note sequence and intensity
reln0=[]
inten0=[]
j=1
while(j<n):
  note= refnote - test1[no][j]
  reln0.append(note)
  inten0.append(test1[no][j+1])
  j=j+4
print("relative notes")
print(reln0)
print("intensity")
print(inten0)

[0.278639, 57, 105, 39, 0.325079, 58, 106, 57, 0.574694, 62, 117, 58, 2.58322, 69, 118, 62, 3.779048, 70, 119, 69, 5.787574, 61, 114, 0]
duration of notes
[0.04644, 0.249615, 2.008526, 1.195828, 2.008526, 0.168095]
117
119
4
70
relative notes
[13, 12, 8, 1, 0, 9]
intensity
[105, 106, 117, 118, 119, 114]


In [11]:
no=1
print(test1[no])
n=len(test1[no])

# to find the reference note find the duration of each note
i=0
j=4
dur1=[]

print("duration of notes")
while(j<n):
  time = test1[no][j]- test1[no][i]
  dur1.append(round(time,6))
  i=i+4
  j=j+4
temp= totd[no]-test1[no][i]

dur1.append(round(temp,6))
print(dur1)
#identify the max duration and corrosponding note
maxd=0
j=1
k=len(dur1)
while j<k:
  if (dur1[maxd]==dur1[j]):
    if (test1[no][maxd*4+2]<test1[no][j*4+2]):
      print(test1[no][maxd*4+2])
      print(test1[no][j*4+2])
      maxd=j
  elif (dur1[maxd]<dur1[j]):
    maxd=j
  j=j+1

print(maxd)
# display reference note
print(test1[no][(maxd)*4+1])
refnote= test1[no][(maxd)*4+1]
# create relative note sequence and intensity
reln1=[]
inten1=[]
j=1
while(j<n):
  note= refnote - test1[no][j]
  reln1.append(note)
  inten1.append(test1[no][j+1])
  j=j+4
print("relative notes")
print(reln1)
print("intensity")
print(inten1)

[0.15093, 61, 88, 0, 0.18576, 60, 81, 61, 0.226395, 58, 83, 60, 0.278639, 58, 93, 58, 0.330884, 60, 94, 58, 0.679184, 66, 99, 60, 3.082449, 72, 105, 66, 5.433469, 72, 107, 0, 6.635102, 62, 98, 0]
duration of notes
[0.03483, 0.040635, 0.052244, 0.052245, 0.3483, 2.403265, 2.35102, 1.201633, 0.138821]
5
66
relative notes
[5, 6, 8, 8, 6, 0, -6, -6, 4]
intensity
[88, 81, 83, 93, 94, 99, 105, 107, 98]


In [12]:
no=2
print(test1[no])
n=len(test1[no])

# to find the reference note find the duration of each note
i=0
j=4
dur2=[]

print("duration of notes")
while(j<n):
  time = test1[no][j]- test1[no][i]
  dur2.append(round(time,6))
  i=i+4
  j=j+4
temp= totd[no]-test1[no][i]

dur2.append(round(temp,6))
print(dur2)
#identify the max duration and corrosponding note
maxd=0
j=1
k=len(dur2)
while j<k:
  if (dur2[maxd]==dur2[j]):
    if (test1[no][maxd*4+2]<test1[no][j*4+2]):
      print(test1[no][maxd*4+2])
      print(test1[no][j*4+2])
      maxd=j
  elif (dur2[maxd]<dur2[j]):
    maxd=j
  j=j+1

print(maxd)
# display reference note
print(test1[no][(maxd)*4+1])
refnote= test1[no][(maxd)*4+1]
# create relative note sequence and intensity
reln2=[]
inten2=[]
j=1
while(j<n):
  note= refnote - test1[no][j]
  reln2.append(note)
  inten2.append(test1[no][j+1])
  j=j+4
print("relative notes")
print(reln2)
print("intensity")
print(inten2)

[1.927256, 59, 112, -1, 2.30458, 63, 112, 0, 4.806531, 72, 111, 63, 8.504308, 62, 108, 0, 8.655238, 60, 103, 0]
duration of notes
[0.377324, 2.501951, 3.697777, 0.15093, 0.162653]
2
72
relative notes
[13, 9, 0, 10, 12]
intensity
[112, 112, 111, 108, 103]


In [13]:
print (test1[2])

[1.927256, 59, 112, -1, 2.30458, 63, 112, 0, 4.806531, 72, 111, 63, 8.504308, 62, 108, 0, 8.655238, 60, 103, 0]


In [20]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw
# note represents reference notes from midi for seq 1 2 and 3 
print("notes")
print(reln0)
print(reln1)
print(reln2)

# int represents intensity
print("intensity")
print(inten0)
print(inten1)
print(inten2)

# dur represents duration
print("duration")
print(dur0)
print(dur1)
print(dur2)

print("comparision of seq 0 and 1")
x = np.array(reln0)
y = np.array(reln1)
distance, path = fastdtw(x, y, dist=euclidean)
d1 =distance
print(d1)
x = np.array(inten0)
y = np.array(inten1)
distance, path = fastdtw(x, y, dist=euclidean)
d2=distance
print(d2)
x = np.array(dur0)
y = np.array(dur1)
distance, path = fastdtw(x, y, dist=euclidean)
d3= distance
print(d3)
d=d1+d2+d3
print("total distance")
print(d)

print("comparision of seq 0 and 2")
x = np.array(reln0)
y = np.array(reln2)
distance, path = fastdtw(x, y, dist=euclidean)
d1 =distance
print(d1)
x = np.array(inten0)
y = np.array(inten2)
distance, path = fastdtw(x, y, dist=euclidean)
d2=distance
print(d2)
x = np.array(dur0)
y = np.array(dur2)
distance, path = fastdtw(x, y, dist=euclidean)
d3= distance
print(d3)
d=d1+d2+d3
print("total distance")
print(d)

print("comparision of seq 1 and 2")
x = np.array(reln1)
y = np.array(reln2)
distance, path = fastdtw(x, y, dist=euclidean)
d1 =distance
print(d1)
x = np.array(inten1)
y = np.array(inten2)
distance, path = fastdtw(x, y, dist=euclidean)
d2=distance
print(d2)
x = np.array(dur1)
y = np.array(dur2)
distance, path = fastdtw(x, y, dist=euclidean)
d3= distance
print(d3)
d=d1+d2+d3
print("total distance")
print(d)

notes
[13, 12, 8, 1, 0, 9]
[5, 6, 8, 8, 6, 0, -6, -6, 4]
[13, 9, 0, 10, 12]
intensity
[105, 106, 117, 118, 119, 114]
[88, 81, 83, 93, 94, 99, 105, 107, 98]
[112, 112, 111, 108, 103]
duration
[0.04644, 0.249615, 2.008526, 1.195828, 2.008526, 0.168095]
[0.03483, 0.040635, 0.052244, 0.052245, 0.3483, 2.403265, 2.35102, 1.201633, 0.138821]
[0.377324, 2.501951, 3.697777, 0.15093, 0.162653]
comparision of seq 0 and 1
34.0
142.0
1.7069140000000007
total distance
177.706914
comparision of seq 0 and 2
7.0
47.0
3.969999
total distance
57.969999
comparision of seq 1 and 2
42.0
122.0
3.8783439999999993
total distance
167.878344
